# Introduction



                
        This Notebook Demonstrates synchronizing the system clock and GEM PTP Hardware clock(Both master and slaves)with
        GPS Time using Pmod GPS receiver. 
        ubuntu Deamons(Gpsd,Chrony) and linux ptp utilities (phc2sys,ptp4l) are used for Synchronization.
              
  ### KR260 Boards Test setup            
          
 <img src="gps.png" style="width:900px;height:450px" align="left"/>

 ### Clock Synchronization Flow chart
   <img src="sync_flow.png" style="width:900px;height:450px" align="left"/>

#### Default System time and PHC Clock

In [ ]:
!timedatectl
!phc_ctl /dev/ptp1 get

## Gpsd:
            Gpsd is a service Deamon for establishing the connection to the external GPS Receiver,This Deamon is configured to read and process UBX protocol messages through the Uart interface on the master node.The results are provided via an IPC shared memory buffer

In [ ]:
import os
get_ipython().system = os.system
!mv /dev/ttyULR0 /dev/tty16
!systemctl disable gpsd.socket
!sleep 1
!systemctl stop gpsd.service
!sleep 3
!systemctl start gpsd.service
!killall -9 gpsd
!gpsd -n -N /dev/tty16 & 
!sleep 2
!echo "gpsd service started"

## Chrony:
          Chrony implements a robust time syncronization b/w the system clock with GPS,it is configured to work with shared memory segment and also process the PPS (/dev/ppsXX) interface.

In [ ]:
!timedatectl

In [ ]:
!echo "chrony started"
!systemctl start chrony
!sleep 180
!chronyc sources
!chronyc tracking
!echo "************** System Clock Syncronized With GPS Time****************"
!timedatectl

## Phc2sys:
            phc2sys establishes the synchronization between the PHC and system clocks – the actual synchronization direction depends on the role of the node

In [ ]:
!timedatectl
!phc_ctl /dev/ptp1 get

In [ ]:
!echo "*********************** Phc2sys Running *****************************"
!phc2sys -c /dev/ptp1 -s CLOCK_REALTIME -O 0 &
!sleep 5
!echo "*************** PHC clock Syncronized with system clock***************"
!phc_ctl /dev/ptp1 get
!timedatectl

## ptp4l:
        ptp4l implements the PTP network protocol and aligns the PHC clocks on the slave side

In [ ]:
!ptp4l -i eth0  &
!echo "*************** Ptp4l Application is Running***************"

In [ ]:
!timedatectl
!phc_ctl /dev/ptp1 get